# O-RAN O2 DMS 自動化部署腳本 (OAI 5G CU)

本 Notebook 演示如何透過 O2 DMS REST API 自動化部署 **OpenAirInterface (OAI) CU**。
流程包含以下步驟：
1. **Configuration**: 設定全域變數 (IPs, Cluster UUID)。
2. **Create VNF Descriptor**: 向 O-Cloud 註冊 Helm Chart 資訊與 VNF 參數。
3. **Create Deployment**: 建立部署物件並綁定 Descriptor。
4. **Instantiate**: 觸發實際的 K8s Pod 部署。
5. **Terminate**: (可選) 清理與釋放資源。

## Cell 2: [Code] 引入套件與參數設定
說明： 將 IP、Cluster UUID 和 Headers 提取出來，方便統一管理，不用每次改 Code 都要翻很多行。

In [ ]:
import requests
import json
import time

# --- Configuration ---
O2_DMS_HOST = "192.168.8.35"
O2_DMS_PORT = "8080"
BASE_URL = f"http://{O2_DMS_HOST}:{O2_DMS_PORT}/api/o2dms/v2"

# Target Environment
TARGET_CLUSTER_ID = "0b9afde5-d8bb-4f91-956b-fdd424e5b766"
TARGET_NAMESPACE = "johnson-ns"
AMF_IP = "192.168.8.108"

# Common Headers
HEADERS = {
    "Content-Type": "application/json",
    "User-Agent": "insomnia/11.3.0"
}

def print_json(data):
    """Helper function to print JSON prettily"""
    print(json.dumps(data, indent=2))

: 

## Cell 3: [Code] 步驟一：建立 VNF Descriptor
說明： 定義 OAI CU 的 Helm Chart 來源與 values.yaml 的覆寫參數 (如 AMF IP)。

In [ ]:
# API Endpoint
url = f"{BASE_URL}/vnf_instances/"

# Payload: Define the Artifact (Helm Chart) and Profile
payload = {
    "name": "oai-cu-test-01",
    "description": "OAI CU VNF Descriptor",
    "profile_type": "kubernetes",
    "artifact_repo_url": "https://gitlab.eurecom.fr/oai/orchestration/charts.git",
    "artifact_name": "oai-5g-ran/oai-cu",
    "artifact_repo_branch": "main",
    "target_cluster": TARGET_CLUSTER_ID,
    "namespace": TARGET_NAMESPACE,
    "values": {
        "config.amfHost": AMF_IP,
        "serviceAccount.create": "false",
        "serviceAccount.name": "default"
    }
}

# Request
response = requests.post(url, json=payload, headers=HEADERS)

# Error Handling & Output
if response.status_code in [200, 201]:
    descriptor_data = response.json()
    descriptor_id = descriptor_data.get('descriptor_id')
    print(f"✅ VNF Descriptor Created! ID: {descriptor_id}")
    # print_json(descriptor_data) # Uncomment to see full details
else:
    print(f"❌ Failed to create descriptor. Status: {response.status_code}")
    print(response.text)

Cell 4: [Code] 步驟二：建立 Deployment 物件
說明： 利用上一步驟回傳的 descriptor_id 來建立一個部署紀錄。

In [ ]:
# API Endpoint
url = f"{BASE_URL}/deployments/"

# Payload: Bind the Descriptor ID
payload = {
    "descriptor": descriptor_id,  # From previous cell
    "name": "post-test-01",
    "namespace": TARGET_NAMESPACE
}

# Request
response = requests.post(url, json=payload, headers=HEADERS)

# Error Handling & Output
if response.status_code in [200, 201]:
    deployment_data = response.json()
    instance_id = deployment_data.get('instance_id')
    print(f"✅ Deployment Object Created! Instance ID: {instance_id}")
    # print_json(deployment_data)
else:
    print(f"❌ Failed to create deployment. Status: {response.status_code}")
    print(response.text)

Cell 5: [Code] 步驟三：執行實例化 (Instantiate)
說明： 這是實際觸發 Helm Install 的步驟。我們設定了 replicas: 2。

In [ ]:
if 'instance_id' in locals():
    # API Endpoint
    url = f"{BASE_URL}/deployments/{instance_id}/instantiate/"

    # Payload: Runtime parameters
    payload = {
        "instantiation_params": {
            "replicas": 2, 
            "namespace": TARGET_NAMESPACE
        }
    }

    # Request
    print(f"🚀 Triggering Instantiation for ID: {instance_id}...")
    response = requests.post(url, json=payload, headers=HEADERS)

    print(f"Response Status: {response.status_code}")
    print(response.text)
else:
    print("⚠️ instance_id not found. Please run previous cells first.")

Cell 6: [Code] 步驟四：終止與清理 (Terminate)
說明： 如果測試完畢，執行此 Cell 可以優雅地移除資源。預設設為 graceful: True

In [ ]:
# User Confirmation (Optional safety check)
confirm = input(f"Are you sure you want to TERMINATE instance {instance_id}? (y/n): ")

if confirm.lower() == 'y' and 'instance_id' in locals():
    url = f"{BASE_URL}/deployments/{instance_id}/terminate/"

    payload = {
        "graceful": True,
        "cleanup_namespace": True
    }

    print(f"🛑 Terminating Instance {instance_id}...")
    response = requests.post(url, json=payload, headers=HEADERS)
    
    print(response.text)
else:
    print("Operation cancelled.")